<a href="https://colab.research.google.com/github/moustafa-7/PyTorch-Course/blob/master/Simple_RNN_Classifier_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [0]:
train_dset = dsets.MNIST('./root', download = True, train = True, transform = transforms.ToTensor())
test_dset = dsets.MNIST('./root', download = True, train = False, transform = transforms.ToTensor())

In [5]:
batch_size = 100
iters = 3000
epochs = iters / (len(train_dset) / batch_size)
epochs = int(epochs)
epochs

5

In [0]:
train_loader = torch.utils.data.DataLoader(train_dset,
                                          shuffle = True,
                                          batch_size = batch_size)

test_loader = torch.utils.data.DataLoader(test_dset,
                                         shuffle = True,
                                         batch_size = 100)

In [0]:
  

class rnn_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(rnn_model, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        
        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [0]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

model = rnn_model(input_dim, hidden_dim, layer_dim, output_dim)

In [0]:
criterion = nn.CrossEntropyLoss()

In [0]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [51]:
seq_dim = 28 

iteration = 0
model.cuda()

for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    
    iteration += 1
    
    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
    
    labels = Variable(labels.cuda())
    
    optimizer.zero_grad()
    
    outputs = model(images)
    
    loss = criterion(outputs, labels)
    
    loss.backward()
    
    optimizer.step()
    
    if(iteration%500 == 0):
      correct = 0
      total = 0
      for images, labels in test_loader:
        images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels.cpu()).sum()
        
      accuracy = 100 * float(correct) / total
      print('Iteration: {}, Loss: {}, Accuracy: {}'.format(iteration, loss, accuracy))
      
        

Iteration: 500, Loss: 1.4499502182006836, Accuracy: 44.8
Iteration: 1000, Loss: 0.41707274317741394, Accuracy: 84.77
Iteration: 1500, Loss: 0.17758098244667053, Accuracy: 90.73
Iteration: 2000, Loss: 0.13307605683803558, Accuracy: 92.72
Iteration: 2500, Loss: 0.3865855038166046, Accuracy: 90.8
Iteration: 3000, Loss: 0.37381216883659363, Accuracy: 91.43
